In [24]:
import copy
import os
from xml.dom import minidom as dom
from tqdm import tqdm
import cv2
import re

In [15]:
class XMLGenerator(object):
    def __init__(self, xml_name: str):
        self.doc = dom.Document()
        self.xml_name = xml_name

    def create_append_node(self, node_name, root_node=None):
        """创建一个新node并将node添加到root_node下"""
        new_node = self.doc.createElement(node_name)
        if root_node is not None:
            root_node.appendChild(new_node)
        else:
            self.doc.appendChild(new_node)
        return new_node

    def create_text_node(self, node_name, node_value, root_node):
        """
        创建一个新node，然后在该node中添加一个text_node，
        最后将node添加到root_node下
        """
        new_node = self.doc.createElement(node_name)
        node_data = self.doc.createTextNode(node_value)
        new_node.appendChild(node_data)
        root_node.appendChild(new_node)

    def create_object_node(self, info_dict: dict = None, root_node: str = None):
        if (info_dict is None) or (root_node is None):
            return

        object_node = self.create_append_node('object', root_node)
        box_node = self.create_append_node('bndbox', object_node)
        self.create_text_node("xmin", info_dict.pop("xmin"), box_node)
        self.create_text_node("ymin", info_dict.pop("ymin"), box_node)
        self.create_text_node("xmax", info_dict.pop("xmax"), box_node)
        self.create_text_node("ymax", info_dict.pop("ymax"), box_node)

        for k, v in info_dict.items():
            self.create_text_node(k, v, object_node)

    def save_xml(self):
        f = open(self.xml_name, "w")
        self.doc.writexml(f, addindent="\t", newl="\n")
        f.close()



In [16]:
def create_pascal_voc_xml(filename: str = None,
                          years: str = 'VOC2012',
                          source_dict: dict = None,
                          objects_list: list = None,
                          im_shape: tuple = None,
                          save_root: str = os.getcwd(),
                          cover: bool = False):
    if not (filename and source_dict and objects_list and im_shape):
        return

    # 0--Parade/0_Parade_marchingband_1_849.jpg -> 0_Parade_marchingband_1_849.xml
    xml_name = filename.split(os.sep)[-1].split(".")[0] + '.xml'
    xml_full_path = os.path.join(save_root, xml_name)
    if os.path.exists(xml_full_path) and (cover is False):
        print(f"{xml_full_path} already exist, skip.")
        return

    xml_generator = XMLGenerator(xml_full_path)

    # xml root node
    node_root = xml_generator.create_append_node('annotation')
    xml_generator.create_text_node(node_name='folder', node_value=years, root_node=node_root)
    xml_generator.create_text_node(node_name='filename', node_value=filename, root_node=node_root)

    # source
    node_source = xml_generator.create_append_node('source', root_node=node_root)
    xml_generator.create_text_node(node_name='database', node_value=source_dict['database'], root_node=node_source)
    xml_generator.create_text_node(node_name='annotation', node_value=source_dict['annotation'], root_node=node_source)
    xml_generator.create_text_node(node_name='image', node_value=source_dict['image'], root_node=node_source)

    # size
    node_size = xml_generator.create_append_node('size', root_node=node_root)
    xml_generator.create_text_node(node_name='height', node_value=str(im_shape[0]), root_node=node_size)
    xml_generator.create_text_node(node_name='width', node_value=str(im_shape[1]), root_node=node_size)
    xml_generator.create_text_node(node_name='depth', node_value=str(im_shape[2]), root_node=node_size)

    # segmented
    xml_generator.create_text_node(node_name='segmented', node_value='0', root_node=node_root)

    # object
    for i, ob in enumerate(objects_list):
        xml_generator.create_object_node(info_dict=ob, root_node=node_root)

    # XML write
    xml_generator.save_xml()




In [17]:
def create_xml_test():
    objects = []
    ob = {'name': 'person', 'pose': 'Unspecified', 'truncated': '0', 'difficult': '0',
          'xmin': '174', 'ymin': '101', 'xmax': '349', 'ymax': '351'}
    objects.append(ob)
    objects.append(copy.deepcopy(ob))

    years = 'VOC2012'
    filename = 'test.jpg'
    source_dict = {'database': 'The VOC2007 Database', 'annotation': 'PASCAL VOC2007', 'image': 'flickr'}
    im_width = '500'
    im_height = '700'
    im_depth = '3'
    im_shape = (im_width, im_height, im_depth)
    create_pascal_voc_xml(filename=filename, years=years,
                          source_dict=source_dict, objects_list=objects,
                          im_shape=im_shape)

In [18]:
def create_xml(labels: list, img_root: str, img_path: str, save_root: str) -> bool:
    source_dict = {'database': 'The WIDERFACE2017 Database',
                   'annotation': 'WIDERFACE 2017',
                   'image': 'WIDERFACE'}

    img_name = img_path.split("/")[1].split(".")[0]
    n_img_dir = os.path.join(save_root, img_name + '.jpg')
    img_full_path = os.path.join(img_root, img_path)
    if os.path.exists(img_full_path):
        im = cv2.imread(img_full_path)
        im_shape = im.shape
        cv2.imwrite(n_img_dir, im)
    else:
        print(f"Warning: {img_path} does not exist, can't read image shape.")
        im_shape = (0, 0, 0)

    ob_list = []
    for ob in labels:
        if ob[7] == '1':
            # invalid face image, skip
            continue

        if int(ob[2]) <= 0 or int(ob[3]) <= 0:
            print(f"Warning: find bbox w or h <= 0, in {img_path}, skip.")
            continue

        ob_dict = {'name': 'face',
                   'truncated': '0' if ob[8] == '0' else '1', # 人脸的遮挡程度
                   'difficult': '1' if ob[4] == '2' or ob[8] == '2' else '0', # 很模糊或遮挡严重为1
                   'xmin': ob[0], 'ymin': ob[1],
                   'xmax': str(int(ob[0]) + int(ob[2])),
                   'ymax': str(int(ob[1]) + int(ob[3])),
                   'blur': ob[4], 'expression': ob[5],
                   'illumination': ob[6], 'invalid': ob[7],
                   'occlusion': ob[8], 'pose': ob[9]}

        # if ob[7] == '1':
        #     cv2.rectangle(im, (int(ob_dict['xmin']), int(ob_dict['ymin'])),
        #                   (int(ob_dict['xmax']), int(ob_dict['ymax'])),
        #                   (0, 0, 255))
        #     cv2.imshow("s", im)
        #     cv2.waitKey(0)

        ob_list.append(ob_dict)
    
    if len(ob_list) == 0: 
        print(f"in {img_path}, no object, skip.")
        return False

    create_pascal_voc_xml(filename=img_path,
                          years="WIDERFACE2017",
                          source_dict=source_dict,
                          objects_list=ob_list,
                          im_shape=im_shape,
                          save_root=save_root)

    return True



In [19]:
def parse_wider_txt(data_root: str, split: str, save_root: str):
    """
    refer to: torchvision.dataset.widerface.py
    :param data_root:
    :param split:
    :param save_root:
    :return:
    """
    assert split in ['train', 'val'], f"split must be in ['train', 'val'], got {split}"

    if os.path.exists(save_root) is False:
        os.makedirs(save_root)

    txt_path = os.path.join(data_root, 'wider_face_split', f'wider_face_{split}_bbx_gt.txt')
    img_root = os.path.join(data_root, f'WIDER_{split}', 'images')
    
    with open(txt_path, "r") as f:
        lines = f.readlines()
        file_name_line, num_boxes_line, box_annotation_line = True, False, False
        num_boxes, box_counter, idx = 0, 0, 0
        labels = []
        xml_list = []
        progress_bar = tqdm(lines)
        for line in progress_bar:
            line = line.rstrip()
            if file_name_line:
                img_path = line
                file_name_line = False
                num_boxes_line = True
            elif num_boxes_line:
                num_boxes = int(line)
                num_boxes_line = False
                box_annotation_line = True
            elif box_annotation_line:
                box_counter += 1
                line_split = line.split(" ")
                line_values = [x for x in line_split]
                labels.append(line_values)
                if box_counter >= num_boxes:
                    box_annotation_line = False
                    file_name_line = True

                    if num_boxes == 0:
                        print(f"in {img_path}, no object, skip.")
                    else:
                        if create_xml(labels, img_root, img_path, save_root):
                            # 只记录有目标的xml文件
                            xml_list.append(img_path.split("/")[-1].split(".")[0])

                    box_counter = 0
                    labels.clear()
                    idx += 1
                    progress_bar.set_description(f"{idx} images")
            else:
                raise RuntimeError("Error parsing annotation file {}".format(txt_path))

        with open(split+'.txt', 'w') as w:
            w.write("\n".join(xml_list))




In [21]:
# parse_wider_txt("../../../data/WiderFace/", "train", "../../../data/WiderFace/train/")

285 images:   6%|█▎                    | 10623/185184 [00:08<02:41, 1082.62it/s]

in 0--Parade/0_Parade_Parade_0_452.jpg, no object, skip.


1159 images:  17%|███▋                  | 31019/185184 [00:30<03:25, 748.82it/s]

2988 images:  30%|██████▎              | 55659/185184 [01:16<01:05, 1975.72it/s]

3108 images:  32%|██████▋              | 59337/185184 [01:19<01:49, 1153.74it/s]

...

9557 images:  82%|█████████████████▏   | 151308/185184 [04:36<00:42, 789.60it/s]

9589 images:  82%|█████████████████▏   | 151511/185184 [04:36<01:03, 534.30it/s]

10976 images:  87%|█████████████████▍  | 161878/185184 [05:12<00:43, 535.58it/s]

12389 images:  96%|██████████████████▎| 178038/185184 [05:47<00:04, 1670.73it/s]

12880 images: 100%|████████████████████| 185184/185184 [05:59<00:00, 515.23it/s]


In [22]:
def get_filename(root_dir = "../../../data/WiderFace/train/"):
    name_list = []
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            file = file.strip()
            if file != '.DS_Store' and file != 'filename.txt' and file.endswith(".xml"):
                file = re.sub(r'.(xml|jpg|png)$', "", file)
                name_list.append(file + '\n')
    print(len(name_list), sep='\n')
    name_list = list(set(name_list))
    all_len = len(name_list)
    print(all_len, sep='\n')


    with open(root_dir + '/filename.txt', 'w') as f:
        f.writelines(name_list)



In [27]:
# get_filename(root_dir = "../../../data/WiderFace/train/")

12876
12876


In [32]:
def read_lines(root_dir = "../../../data/WiderFace/train/"):
    with open(root_dir + 'filename.txt', 'r') as f:
        l = f.readlines()
        print(len(l))

In [33]:
# read_lines()

12876
